In [1]:
!pip install magenta
!pip install note_seq
!pip install --upgrade tensorflow

     |████████████████████████████████| 1.4 MB 5.4 MB/s 
     |████████████████████████████████| 204 kB 46.4 MB/s 
     |████████████████████████████████| 3.6 MB 15.1 MB/s 
     |████████████████████████████████| 254 kB 36.2 MB/s 
     |████████████████████████████████| 90 kB 9.4 MB/s 
     |████████████████████████████████| 1.4 MB 36.0 MB/s 
     |████████████████████████████████| 69 kB 5.3 MB/s 
     |████████████████████████████████| 352 kB 36.0 MB/s 
     |████████████████████████████████| 2.3 MB 26.2 MB/s 
     |████████████████████████████████| 5.6 MB 22.4 MB/s 
     |████████████████████████████████| 1.6 MB 35.3 MB/s 
     |████████████████████████████████| 210 kB 31.2 MB/s 
     |████████████████████████████████| 20.2 MB 1.7 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 5.8 MB 28.1 MB/s 
     |████████████████████████████████| 193 kB 44.8 MB/s 
     |████████████████████████████████| 981 kB 42.5 MB/s 
     |███████████████

## A. Data 및 metadata 분석(실제와 차이가 있는것으로 보임, 참고용)

### dataset 설명
- 10명의 드러머들이 다른 스타일로 쳤음
- 각각 **장르, 템포, 익명의 드러머 ID 로 annotate되어있음**
- 대부분 4/4 박자(아닌것도 있음)
- 4명의 드러머는 그들의 style로 10 비트의 같은 set들을 연주하도록 요청받음 → eval-session/groove1-10 test set에 있음
- 13.6 시간
- 1,150개 midi file
- 22,000 drumming
- midi 데이터로부터 합성된 audio output이 추가로 있음
- MIDI와 WAV 파일, CSV 포맷

### Data Format

여러 개의 Track으로 나뉘어짐

1. meta-messages(템포, 시간,key)
2. control changes(hi-hat pedal position)
    - channel 0 에 설정되어 있음
    - control 4로 유지되었음
3. notes(음표)
    - channel 9 에 설정되어 있음(드럼 표준 채널)

**→ 단순화하기 위하여 3 트랙을 하나의 트랙으로 병합**

**→ 모든 메시지를 channel 9 로 설정**


### folder 확인
- 10명의 drummer에 대하여
  - 각 `session`에 대하여
    - `session` 별 `id`(performance)가 존재 (midi file 단위)
- 각 메타데이터는 폴더 경로가 순차적으로 적혀 있음

### id별 특성
- `style` : `pop-groove1` 과 같은 형식, `str` type
- `bpm` : beat per minute (`int` type)
- `beat_type` : `beat` 또는 `fill`
- `time_signature` : `4-4` 와 같은 형식(4/4 박자를 의미)
- `midi_filename` : 파일 상대경로
- `duration` : 길이 (`float` type)
- `split` : `train`/`valid`/`test`로 나뉘어 있음

> id_style_bpm_beat_type_time_signature







  

In [2]:
import pandas as pd
path = '/content/drive/MyDrive/MusicVAE_project/groove/info.csv'
metadf = pd.read_csv(path)
metadf.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test


In [3]:
metadf.sort_values(by='split')[['split', 'midi_filename']]

,split,midi_filename
0,test,drummer1/eval_session/1_funk-groove1_138_beat_...
109,test,drummer1/session1/189_reggae_78_fill_4-4.mid
903,test,drummer7/session3/133_soul_105_beat_4-4.mid
902,test,drummer7/session3/132_soul_105_beat_4-4.mid
758,test,drummer7/session2/122_jazz-fusion_96_fill_4-4.mid
...,...,...
278,validation,drummer1/session2/1_latin-chacarera_157_beat_3...
277,validation,drummer1/session1/99_neworleans-funk_84_fill_4...
904,validation,drummer7/session3/134_soul_105_fill_4-4.mid
914,validation,drummer7/session3/143_soul_105_fill_4-4.mid


- metadata와 실제 데이터 명이 부합하지 않음 -> 비율만 가져가고 임의로 자르는 것으로 결정

In [4]:
# 900:125:125 = 36:5:5
metadf.groupby('split').count()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration
split,,,,,,,,,,
test,129,129,129,129,129,129,129,129,124,129
train,897,897,897,897,897,897,897,897,846,897
validation,124,124,124,124,124,124,124,124,120,124


## B. EDA
- midi file -> vector화 -> `tfrecord`

### 해결해야 할 과제

- midi file을 어떻게 불러오고 tfrecord로 어떻게 변환할 것인가?
  1. `convert_midi` 로 sequence 할당
  2. `sequence`를 앞에서부터 4마디에 해당하는 데이터로 trim
  3. `GrooveConverter`의 `to_tensor` 메소드를 통해 `hits`, `velocities`, `offsets` 를 포함한 one-hot tensor 반환
  4. tensor를 `dataset`화
  5. `train`/`validation`/`test` split
  5. `get_input_tensor` 를 통해 `input`, `output`, `control`, `length` 에 해당하는 데이터를 dictionary로 반환

-  musicVAE를 어떻게 구현할 것인가?
  - `train`, `evaluate`, `generate` 를 통하여 모델을 학습 및 검증


### 1) 하나의 데이터에 대하여 추출 및 분석
 - 하나의 데이터를 확인하여 이해도를 높인 후 전체 데이터셋에 적용

In [5]:
%cd /content/drive/MyDrive/MusicVAE_project
from magenta.scripts.convert_dir_to_note_sequences import * 
# 전처리 과정 예시
root_dir = '/content/drive/MyDrive/MusicVAE_project/'
sub_dir = '/content/drive/MyDrive/MusicVAE_project/groove/drummer1/eval_session/'
full_file_path='/content/drive/MyDrive/MusicVAE_project/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid'



seq = convert_midi(root_dir, sub_dir, full_file_path)

seq

/content/drive/MyDrive/MusicVAE_project


/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE_project/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid.


id: "/id/midi//e12bcb0c4f20da0575128237ac1329cfc28259b7"
filename: "/content/drive/MyDrive/MusicVAE_project/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid"
ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 137.99987580011177
}
notes {
  pitch: 55
  velocity: 60
  start_time: 0.00271739375
  end_time: 0.10416676041666667
  is_drum: true
}
notes {
  pitch: 51
  velocity: 55
  start_time: 0.006340585416666667
  end_time: 0.10688415416666668
  is_drum: true
}
notes {
  pitch: 36
  velocity: 64
  start_time: 0.009057979166666667
  end_time: 0.10869575000000001
  is_drum: true
}
notes {
  pitch: 36
  velocity: 57
  start_time: 0.19112336041666667
  end_time: 0.2916669291666667
  is_drum: true
}
notes {
  pitch: 51
  velocity: 31
  start_time: 0.21557990416666667
  end_time: 0.31521767500000003
  is_drum: true
}
notes {
  pitch: 44
  velocity: 77
  start_time: 0.4039858708333333
  end_time: 0.5054352375
  is_drum: true


### sequence prototype data analysis(attribute 형태로 접근 가능)
  - `id` : hash value
  - `filename` : 경로를 포함한 파일 이름
  -  `ticks_per_quarter`: 
  - `time_signatures`:4/4박자 정보가 있음
  - `key_signatures`: 공란
  - `notes`: 음들
    - pitch, velocity, start_time, end_time, is_drum
    - pitch는 일반 음 높낮이와 다른 별도의 mapping 시스템을 따름
  - `control_changes` : 하이햇 조절 페달 변화
    - time, control_number, control_value, is_drum
    - control number는 4로 통일되어 있음


In [6]:
seq.notes[0]

pitch: 55
velocity: 60
start_time: 0.00271739375
end_time: 0.10416676041666667
is_drum: true

In [7]:
seq.control_changes[0]

time: 0.011775372916666667
control_number: 4
control_value: 90
is_drum: true

### TFRecord?
- 데이터 세트의 포맷의 하나로 TFRecord형식은 바이너리 코드의 시리즈를 저장하기 위한 단순한 형식
-  TFRecord를 사용하는 것으로, 대규모 데이터를 효율적으로 학습할 수 있게 된다

출처: https://engineer-mole.tistory.com/206 [매일 꾸준히, 더 깊이]

In [8]:
# # 경로를 넣어주면 train, validation, test 판단해주는 함수
# import os

# def splitvalue(filepath):
#   try:
#     filename = filepath.split('groove/')[-1]
#     print(filename)
#     split=metadf[metadf['midi_filename']==filename]['split'].iloc[0]
#     return split

#   except:
#     return None


# 'train'==splitvalue('/content/drive/MyDrive/MusicVAE_project/groove/drummer2/session2/10_rock_130_beat_4-4.mid')

In [9]:
# root_dir = '/content/drive/MyDrive/MusicVAE_project/groove'
# output_file = '/content/drive/MyDrive/MusicVAE_project/output/dataset.tfrecord'
# convert_directory(root_dir, output_file, recursive=True)

In [10]:
!pip install apache_beam

     |████████████████████████████████| 10.2 MB 5.5 MB/s 
     |████████████████████████████████| 46 kB 2.3 MB/s 
     |████████████████████████████████| 508 kB 47.9 MB/s 
     |████████████████████████████████| 151 kB 46.6 MB/s 
     |████████████████████████████████| 253 kB 46.6 MB/s 
     |████████████████████████████████| 2.3 MB 37.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 1.0 MB 43.7 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=331ab337164417c171e3cd6e059cd5492167b87dd294da2de7b5f4056971cd18
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
Successfully built dill
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling

In [11]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [12]:
# from magenta.models.music_vae import preprocess_tfrecord
# filters = {  # pylint: disable=g-long-ternary
#       'max_total_time': 1800,
#       'max_num_notes': 10000,
#       'min_velocities': 1,
#       'min_metric_positions': 1,
#       'is_drum': True,
#       'drums_only': True,
#   }
# example=preprocess_tfrecord.ExtractExamplesDoFn(config,filters)


In [13]:
# 4마디 추출을 위한 변환 함수
def bar_to_dur(bars, bpm):# number of bar
  beats = 4*bars
  duration = beats*60/bpm 
  return duration

def dur_to_bar(dur,bpm):
  total_beat = dur/60*bpm
  return total_beat/4

In [14]:
# control_time trimming을 위한 customizing
from note_seq import sequences_lib
from note_seq.protobuf import music_pb2
def trim_control_sequence(sequence, start_time,end_time):
  """Trim control_changes from a control_changesequence to lie within a specified time range.
  control_changes starting before `start_time` are not included. control_changes ending after
  `end_time` are truncated.
  Args:
    sequence: The control_changesequence for which to trim control_changes.
    start_time: The float time in seconds after which all control_changes should begin.
    end_time: The float time in seconds before which all control_changes should end.
  Returns:
    A copy of `sequence` with all control_changes trimmed to lie between `start_time` and
    `end_time`.
  Raises:
    QuantizationStatusError: If the sequence has already been quantized.
  """
  subsequence = music_pb2.NoteSequence()
  subsequence.CopyFrom(sequence)

  del subsequence.control_changes[:]
  for control in sequence.control_changes:
    if control.time < start_time or control.time >= end_time:
      continue
    new_control = subsequence.control_changes.add()
    new_control.CopyFrom(control)
    new_control.time = min(control.time, end_time)

  subsequence.total_time = min(sequence.total_time,end_time)

  return subsequence

In [15]:
# 4마디로 자르기
import math
from note_seq.sequences_lib import trim_note_sequence
class CustomExtract:
  def __init__(self, seq) -> None:
    self.seq = seq
    self.bpm = seq.tempos[0].qpm
    self.duration = seq.total_time
    self.time_sig = seq.time_signatures[0].numerator/seq.time_signatures[0].denominator

  def extract_4_bars(self):
      beats = 4*4 
      fourbar_duration = beats*60/self.bpm
      print(f'duration of four bars is : {fourbar_duration}sec')
      seq_fourbar = trim_note_sequence(self.seq, 0, math.floor(fourbar_duration))
      return trim_control_sequence(seq_fourbar, 0, math.floor(fourbar_duration))
      

In [16]:
seq.time_signatures[0]

numerator: 4
denominator: 4

In [17]:
# class sequence
# def bpm_value()
seq.tempos[0].qpm

137.99987580011177

In [18]:
seq.notes[0]

pitch: 55
velocity: 60
start_time: 0.00271739375
end_time: 0.10416676041666667
is_drum: true

In [19]:
seq.control_changes[0]

time: 0.011775372916666667
control_number: 4
control_value: 90
is_drum: true

In [20]:
seq.total_time

27.87230769375

In [21]:
# CustomExtract Class로 4마디 추출
inst = CustomExtract(seq)
if inst.time_sig==4/4:
   seq_fourbar = inst.extract_4_bars()
seq_fourbar

duration of four bars is : 6.9565280000000005sec


id: "/id/midi//e12bcb0c4f20da0575128237ac1329cfc28259b7"
filename: "/content/drive/MyDrive/MusicVAE_project/groove/drummer1/eval_session/1_funk-groove1_138_beat_4-4.mid"
ticks_per_quarter: 480
time_signatures {
  numerator: 4
  denominator: 4
}
key_signatures {
}
tempos {
  qpm: 137.99987580011177
}
notes {
  pitch: 55
  velocity: 60
  start_time: 0.00271739375
  end_time: 0.10416676041666667
  is_drum: true
}
notes {
  pitch: 51
  velocity: 55
  start_time: 0.006340585416666667
  end_time: 0.10688415416666668
  is_drum: true
}
notes {
  pitch: 36
  velocity: 64
  start_time: 0.009057979166666667
  end_time: 0.10869575000000001
  is_drum: true
}
notes {
  pitch: 36
  velocity: 57
  start_time: 0.19112336041666667
  end_time: 0.2916669291666667
  is_drum: true
}
notes {
  pitch: 51
  velocity: 31
  start_time: 0.21557990416666667
  end_time: 0.31521767500000003
  is_drum: true
}
notes {
  pitch: 44
  velocity: 77
  start_time: 0.4039858708333333
  end_time: 0.5054352375
  is_drum: true


In [22]:
from magenta.models.music_vae import data

inst = data.GrooveConverter(split_bars=2) # 2마디별로 쪼갬
tensor = inst.to_tensors(seq_fourbar)
# embedding 차원, 길이
tensor[0][0].shape




(32, 27)

### 3)GrooveConverter output 설명
- input : encoder 로 들어갈 입력값
- output :  decoder로 들어갈 입력값
- control: 쓰지 않음
- lengths : input/output의 길이

Timestep(T)  = 4
instruments(M) = 27

In [23]:
tensor[0][0]

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         5.03937008e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  4.72440945e-01,  4.33070866e-01,
        -1.66666667e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -5.00000000e-02, -1.16666667e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+

### input에 해당될 값 설명
- `Hit`: 드럼을 친지 안친지 여부. 1또는 0 의 binary한 값
- `Offset`: `-0.5` ~` 0.5` 의 값
- `velocity`: 드럼치는 속도. `0~127` 값이 `0~1` 사이로 매핑됨


## C. Preprocessing (전체 데이터셋에 적용)
1. 전체 데이터셋을 train,valid,test 900:125:125 로 나눔
  - metadata의 dataset split 비율을 그대로 적용
2. 4/4 박자인 경우에만 seq에 대해 4마디 추출
  - 4/4 박자가 아닌 데이터는 1150개중 12개
3. `GrooveConverter` 활용하여 2마디를 단위로 ConverterTensor 추출함.`(32,27)`
  - 논문에서는 마디단위로 추출하는 것이 좋다고 나오며, 2마디를 default로 전처리를 진행함.


### 1) 여러 시행착오 briefing
1. metadata를 활용하여 경로에 따라 train,validation,test를 판단해주는 함수를 구현하여 활용하려 했으나, metadata와 실제 파일명이 다른 것을 확인 
  -> 미사용

2. `convert_directory`를 이용하여 한번에 tfrecord를 기록하여 이후 전처리 시도
  -> tfrecord 자료형에 대한 이해 부족, `preprocess_tfrecord` 모듈의 활용 방안에 대한 미흡한 검토로 인해 적극적으로 활용하지 못한 방법

3. customizing을 결심하고 데이터 디렉토리 구조에 따라 하나하나 sequence로 할당하고, 4마디를 추출하여 GrooveConverter를 통해 학습에 필요한 전처리를 진행하는 코드를 직접 구현
  - `trim_converter_sequence` 로 note들을 4마디에 해당하는 duration으로 변환 후 trim
  - control_changes 를 위 함수를 customizing하여 마찬가지로 구현(반드시 필요한 작업인지는 검토 필요)

4. 학습을 위해 모듈을 활용하여야 하는데, 

  1) GrooveConverter 로 ConverterTensor로 변환 하는 작업

  2) **ConverterTensor-> Dataset** 변환작업

  3) dataset으로부터 tensor 데이터를 배치화 및 정제하는 작업

2번 과정에서 거듭된 디버깅 실패로 학습을 진행하지 못했음.




In [24]:
from magenta.models.music_vae import data

cnt =0 
results=[]
for i in range(1,11):
  root_dir = f'/content/drive/MyDrive/MusicVAE_project/groove/drummer{i}'
  subdir_list = [x[0] for x in os.walk(root_dir)][1:]
  for subdir in subdir_list:
    print(subdir)
    midifiles = [x for x in os.walk(subdir)][0][2]
    print(midifiles)
    for midi in midifiles:
      cnt+=1
      # print('hi')
      print(midi)
      filename = os.path.join(subdir,midi)
      seq = convert_midi(root_dir, subdir, filename)
      inst = CustomExtract(seq)
      if inst.time_sig==4/4:
        seq_fourbar = inst.extract_4_bars()
        instan = data.GrooveConverter(split_bars=2) # 2마디별로 쪼갬
        tensor = instan.to_tensors(seq_fourbar) 
      else:
        print('pass')
        continue
      # output_dir = '/content/drive/MyDrive/MusicVAE_project/output'
      # if cnt<900:
        
      #   output_file = os.path.join(output_dir, 'train.tfrecord')
        
      # elif cnt<1025:
      #   output_file = os.path.join(output_dir, 'valid.tfrecord')
      # else:
      #   output_file = os.path.join(output_dir, 'test.tfrecord')

      
      # with tf.io.TFRecordWriter(output_file) as writer:
      # print(tensor.shape)
      results.append(tensor)


     

print(cnt)
      
      # print(filename)
      # print(splitvalue(filename))
#   sub_dir = f'/content/drive/MyDrive/MusicVAE_project/groove/drummer{i}'
#   output_file = '/content/drive/MyDrive/MusicVAE_project/output/example.tfrecord'
#   output_dir = '/content/drive/MyDrive/MusicVAE_project/output/'
#   with tf.io.TFRecordWriter(output_file) as writer:
#     # print(writer)
#     cus_convert_files(root_dir, sub_dir, writer,output_dir, recursive=True)

# files_in_dir = tf.gfile.ListDirectory(os.path.join(dir_to_convert))

/content/drive/MyDrive/MusicVAE_project/groove/drummer1/session3
['1_rock-prog_125_beat_4-4.mid', '10_rock-folk_90_beat_4-4.mid', '11_afrobeat_94_beat_4-4.mid', '12_latin-bomba_96_beat_4-4.mid', '13_rock_115_beat_4-4.mid', '14_rock_94_beat_4-4.mid', '15_funk_117_beat_4-4.mid', '16_funk_117_beat_4-4.mid', '18_rock_128_beat_4-4.mid', '17_afrocuban-calypso_108_beat_4-4.mid', '2_jazz-swing_185_beat_4-4.mid', '3_rock-rockabilly_200_beat_4-4.mid', '4_jazz-linear_182_beat_4-4.mid', '5_jazz-linear_128_beat_4-4.mid', '6_dance-disco_120_beat_4-4.mid', '7_dance-disco_120_beat_4-4.mid', '8_rock_135_beat_4-4.mid', '9_rock_135_beat_4-4.mid']
1_rock-prog_125_beat_4-4.mid
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE_project/groove/drummer1/session3/1_rock-prog_125_beat_4-4.mid.
duration of four bars is : 7.68sec
10_rock-folk_90_beat_4-4.mid
INFO:tensorflow:Converted MIDI file /content/drive/MyDrive/MusicVAE_project/groove/drummer1/session3/10_rock-folk_90_beat_4-4.mid.
duration 

In [25]:
# 12개 파일은 4/4 박자 아님
len(results)

1138

In [26]:
data_len=len(results)
train_size = int(36/46*data_len)
val_size =int(5/46*data_len)
test_size =int(5/46*data_len)
trains = results[:train_size]
valids = results[train_size:train_size+val_size]
tests = results[-test_size:]
len(trains), len(valids), len(tests)

(890, 123, 123)

### split
- 실험 결과 dataset을 구성한 이후에 해야겠다고 판단

In [27]:
# # converter tensor들을 합침
# train_dataset = data.combine_converter_tensors(trains)
# valid_dataset = data.combine_converter_tensors(valids)
# test_dataset = data.combine_converter_tensors(tests)

In [28]:
# len(train_dataset.lengths),len(valid_dataset.lengths), len(test_dataset.lengths)

In [29]:
# len(train_dataset)

In [30]:
# len(train_dataset[0])

In [31]:
# train_dataset[0][0].shape

### (32,27) 숫자의 의미
- `32`
  - 한 마디에 16개의 note grid * 4마디 이므로 16*4 = 64
  - 마디별로 sliding window 설정 -> 64/4  = 16 
- `27` : 9종류의 drum category에 대하여 hit,velocity, offset

In [32]:
# control 에 해당하는 데이터는 비어있다
tensor[2]

[array([], shape=(32, 0), dtype=float64),
 array([], shape=(32, 0), dtype=float64)]

input /

In [33]:
tensor[0][0]

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        , -0.11666667,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.    

## D. Train
- `_get_input_tensors` : keyword argument 반환
- dataset으로 반환하는 것이 필요함.

```
  return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
  }
```

- `train` :  train_dir, confing, dataset_fn을 인풋으로 받음
```
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
```

### config
- `configs` 모듈로부터 드럼 musicvae, 다시말해 groovae에 해당하는 
 batch size, embedding size 등 많은 hyperparameter 값들이 설정되어 있으며 그것을 적용하는 것이 옳게 보임. 

In [34]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [35]:
from magenta.models.music_vae.music_vae_train import _get_input_tensors,train,evaluate
from magenta.models.music_vae import configs, update_config
config = configs.CONFIG_MAP['cat-drums_2bar_small']
config

Config(model=<magenta.models.music_vae.base_model.MusicVAE object at 0x7f90ccd85150>, hparams=HParams([('batch_size', 512), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('control_preprocessing_rnn_size', [256]), ('dec_rnn_size', [256, 256]), ('decay_rate', 0.9999), ('dropout_keep_prob', 1.0), ('enc_rnn_size', [512]), ('free_bits', 48), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 32), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 1000), ('sampling_schedule', 'inverse_sigmoid'), ('use_cudnn', False), ('z_size', 256)]), note_sequence_augmenter=None, data_converter=<magenta.models.music_vae.data.DrumsConverter object at 0x7f90ccd85190>, train_examples_path=None, eval_examples_path=None, tfds_name=None)

### 1) 전처리 하지 않은 tfrecord 파일로부터 학습 testing

In [36]:
config = update_config(config, {'train_examples_path':'/content/drive/MyDrive/MusicVAE_project/output/dataset.tfrecord'})

In [37]:
whole_dataset = data.get_dataset(config,is_training=True)

INFO:tensorflow:Reading examples from file: /content/drive/MyDrive/MusicVAE_project/output/dataset.tfrecord


In [38]:
data_dic = _get_input_tensors(whole_dataset, config)

In [39]:
# dictonary 형태의 data input
data_dic['input_sequence'].shape

TensorShape([512, 32, 10])

In [40]:
# input 중 하나인 dataset_function에 대한 이해 필요
trainpath = '/content/drive/MyDrive/MusicVAE_project/train'

train(trainpath, config, data.get_dataset)

NotImplementedError: ignored

### 2) 전처리된 result data 재확인 및 dataset 변환을 위한 추가 시도
- `results` : 1138개의 데이터(list)
안에 4칼럼의 Convertertensor
- 안에 각각 input,output(32,27), control, length가 있음


- 구조가 일정하지 못하여 dataset 변환이 어려움
  - 추가적인 모듈을 활용하지 못하고 있다고 판단
  - input,output,length만 따로 추출하려 했지만 기존 magenta library에 적용하기에는 무리가 있었음.

In [41]:
len(results)

1138

In [42]:
import numpy as np
np.array(results).shape
arr = np.array(results)
arr = np.transpose(arr)
arr.tolist()
len(arr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


4

In [43]:
dataset = tf.data.Dataset.from_tensor_slices(results)

Output hidden; open in https://colab.research.google.com to view.

In [44]:
# raw_dataset = tf.data.TFRecordDataset('/content/drive/MyDrive/MusicVAE_project/output/dataset.tfrecord')
# for raw_record in raw_dataset.take(1):
#   example = tf.train.Example()
#   example.ParseFromString(raw_record.numpy())
#   print(example)

### 3) musicVAE paper

### Abstract

- VAE는 latent representation을 의미론적으로 생산해내는 데에 효과적이라고 알려져 있음
- 하지만 **sequential data에는 RNN을 이용한 제한적인 적용만이 가능함**. 이것은 장기적인 구조를 반영 못함
- `hierarchical decoder`
    - sequence input으로부터 output embedding을 출력
    - 이 embeeding을 각각의 sub -sequence를 독립적으로 생성
    - 이 구조는 모델이 latent code를 utilize하도록 장려
    - `posterior collapse` 를 막음
    - `sampling`, `interpolation`, `reconstruction` 에서 더 나은 성능을 보임

## E. Evaluate


```
def evaluate(train_dir,
             eval_dir,
             config,
             dataset_fn,
             num_batches,
             master=''):
  """Evaluate the model repeatedly."""
```

## F. Test(Generate)
